In [4]:
#!/usr/bin/env python3
# Fetch ALL Freshdesk tickets + conversations, using python-dotenv
# Output: JSON Lines file (one ticket per line) -> tickets_full.jsonl

import os, sys, time, json
from typing import Dict, Iterable, List, Optional
from urllib.parse import urlparse

import requests
from dotenv import load_dotenv, find_dotenv

# ---- Load .env (falls back to real env if already set) ----
load_dotenv(find_dotenv(usecwd=True), override=False)

# ---- Helpers ----
def require_env(name: str) -> str:
    v = os.getenv(name, "").strip()
    if not v:
        sys.exit(f"Missing env var: {name}")
    return v

def build_base_url(domain: str) -> str:
    """
    Accepts 'acme', 'acme.freshdesk.com', or 'https://acme.freshdesk.com'
    -> 'https://acme.freshdesk.com/api/v2'
    """
    d = domain.strip()
    if d.lower().startswith("http"):
        parsed = urlparse(d)
        host = (parsed.netloc or parsed.path).strip("/")
    else:
        host = d.strip("/")
    if not host.endswith(".freshdesk.com"):
        host = f"{host}.freshdesk.com"
    return f"https://{host}/api/v2"

DOMAIN   = require_env("FRESHDESK_DOMAIN")
API_KEY  = require_env("FRESHDESK_API_KEY")
TIMEOUT  = int(os.getenv("FRESHDESK_TIMEOUT_SECONDS", "30"))
BASE_URL = build_base_url(DOMAIN)

session = requests.Session()
session.auth = (API_KEY, "X")  # Freshdesk: API key as username, 'X' as password
session.headers.update({"Content-Type": "application/json"})

def _request(method: str, url: str, params: Optional[Dict]=None) -> requests.Response:
    """Basic 429/5xx retry with exponential backoff."""
    backoff = 1
    while True:
        resp = session.request(method, url, params=params, timeout=TIMEOUT)
        if resp.status_code == 429:
            retry_after = int(resp.headers.get("Retry-After", backoff))
            time.sleep(max(1, retry_after))
            backoff = min(backoff * 2, 30)
            continue
        if 500 <= resp.status_code < 600:
            if backoff > 30:
                resp.raise_for_status()
            time.sleep(backoff)
            backoff = min(backoff * 2, 30)
            continue
        resp.raise_for_status()
        return resp

def _paginate_list_tickets(include: str = "requester,company", per_page: int = 100) -> Iterable[Dict]:
    """
    Iterate over ALL tickets using page/per_page.
    include=requester,company embeds requester/company details for each ticket.
    """
    page = 1
    while True:
        params = {"page": page, "per_page": per_page}
        if include:
            params["include"] = include
        resp = _request("GET", f"{BASE_URL}/tickets", params=params)
        items = resp.json() if resp.text else []
        if not items:
            break
        for t in items:
            yield t
        page += 1

def _paginate_conversations(ticket_id: int, per_page: int = 100) -> List[Dict]:
    """
    Fetch all conversations (agent replies, requester replies, and notes).
    """
    out: List[Dict] = []
    page = 1
    while True:
        params = {"page": page, "per_page": per_page}
        resp = _request("GET", f"{BASE_URL}/tickets/{ticket_id}/conversations", params=params)
        items = resp.json() if resp.text else []
        if not items:
            break
        out.extend(items)
        page += 1
    return out

def fetch_all_tickets_with_details(outfile: str = "tickets_full.jsonl") -> int:
    """
    Streams tickets + conversations to a JSONL file.
    Returns count of tickets written.
    """
    count = 0
    with open(outfile, "w", encoding="utf-8") as f:
        for t in _paginate_list_tickets(include="requester,company"):
            tid = t.get("id")
            try:
                t["conversations"] = _paginate_conversations(tid)
            except requests.HTTPError as e:
                # Keep going even if conversations fail on a ticket
                t["conversations_error"] = {
                    "status": getattr(e.response, "status_code", None),
                    "body": (e.response.text[:300] if getattr(e, "response", None) else "")
                }
            f.write(json.dumps(t, ensure_ascii=False) + "\n")
            count += 1
            if count % 100 == 0:
                print(f"Wrote {count} tickets...", file=sys.stderr)
    return count

if __name__ == "__main__":
    try:
        total = fetch_all_tickets_with_details("tickets_full.jsonl")
        print(f"✅ Done. Wrote {total} tickets to tickets_full.jsonl")
    except requests.HTTPError as e:
        print("HTTP error:", getattr(e.response, "status_code", "?"), (e.response.text[:500] if e.response else ""))
    except Exception as e:
        print("Error:", e)


✅ Done. Wrote 7 tickets to tickets_full.jsonl


In [2]:
from simple_freshdesk import fetch_tickets
rows = fetch_tickets(limit=10)
rows[:2]

ModuleNotFoundError: No module named 'simple_freshdesk'

In [6]:
#!/usr/bin/env python3
# Fetch ALL Freshdesk tickets + conversations, and export to CSV

import os, sys, time, json, csv
from typing import Dict, Iterable, List, Optional
from urllib.parse import urlparse

import requests
from dotenv import load_dotenv, find_dotenv

# ---- Load .env (falls back to real env if already set) ----
load_dotenv(find_dotenv(usecwd=True), override=False)

# ---- Helpers ----
def require_env(name: str) -> str:
    v = os.getenv(name, "").strip()
    if not v:
        sys.exit(f"Missing env var: {name}")
    return v

def build_base_url(domain: str) -> str:
    """
    Accepts 'acme', 'acme.freshdesk.com', or 'https://acme.freshdesk.com'
    -> 'https://acme.freshdesk.com/api/v2'
    """
    d = domain.strip()
    if d.lower().startswith("http"):
        parsed = urlparse(d)
        host = (parsed.netloc or parsed.path).strip("/")
    else:
        host = d.strip("/")
    if not host.endswith(".freshdesk.com"):
        host = f"{host}.freshdesk.com"
    return f"https://{host}/api/v2"

# ---- Environment Setup ----
DOMAIN   = require_env("FRESHDESK_DOMAIN")
API_KEY  = require_env("FRESHDESK_API_KEY")
TIMEOUT  = int(os.getenv("FRESHDESK_TIMEOUT_SECONDS", "30"))
BASE_URL = build_base_url(DOMAIN)

session = requests.Session()
session.auth = (API_KEY, "X")  # Freshdesk: API key as username, 'X' as password
session.headers.update({"Content-Type": "application/json"})

# ---- API Helpers ----
def _request(method: str, url: str, params: Optional[Dict]=None) -> requests.Response:
    """Basic 429/5xx retry with exponential backoff."""
    backoff = 1
    while True:
        resp = session.request(method, url, params=params, timeout=TIMEOUT)
        if resp.status_code == 429:
            retry_after = int(resp.headers.get("Retry-After", backoff))
            time.sleep(max(1, retry_after))
            backoff = min(backoff * 2, 30)
            continue
        if 500 <= resp.status_code < 600:
            if backoff > 30:
                resp.raise_for_status()
            time.sleep(backoff)
            backoff = min(backoff * 2, 30)
            continue
        resp.raise_for_status()
        return resp

def _paginate_list_tickets(include: str = "requester,company", per_page: int = 100) -> Iterable[Dict]:
    """Iterate over ALL tickets using page/per_page."""
    page = 1
    while True:
        params = {"page": page, "per_page": per_page}
        if include:
            params["include"] = include
        resp = _request("GET", f"{BASE_URL}/tickets", params=params)
        items = resp.json() if resp.text else []
        if not items:
            break
        for t in items:
            yield t
        page += 1

def _paginate_conversations(ticket_id: int, per_page: int = 100) -> List[Dict]:
    """Fetch all conversations for a ticket."""
    out: List[Dict] = []
    page = 1
    while True:
        params = {"page": page, "per_page": per_page}
        resp = _request("GET", f"{BASE_URL}/tickets/{ticket_id}/conversations", params=params)
        items = resp.json() if resp.text else []
        if not items:
            break
        out.extend(items)
        page += 1
    return out

# ---- Flatten Helpers ----
def flatten_ticket(ticket: Dict) -> Dict:
    return {
        "id": ticket.get("id"),
        "subject": ticket.get("subject"),
        "status": ticket.get("status"),
        "priority": ticket.get("priority"),
        "created_at": ticket.get("created_at"),
        "updated_at": ticket.get("updated_at"),
        "requester_id": ticket.get("requester", {}).get("id"),
        "requester_name": ticket.get("requester", {}).get("name"),
        "requester_email": ticket.get("requester", {}).get("email"),
        "company_name": ticket.get("company", {}).get("name"),
        "num_conversations": len(ticket.get("conversations", []))
    }

def flatten_conversations(ticket_id: int, convos: List[Dict]) -> List[Dict]:
    rows = []
    for c in convos:
        rows.append({
            "ticket_id": ticket_id,
            "conversation_id": c.get("id"),
            "body_text": c.get("body_text", "").replace("\n", " ").replace("\r", " "),
            "created_at": c.get("created_at"),
            "private": c.get("private")
        })
    return rows

# ---- Main Logic to CSV ----
def fetch_all_tickets_to_csv(ticket_csv="tickets.csv", convo_csv="conversations.csv") -> int:
    ticket_rows = []
    convo_rows = []
    count = 0

    for t in _paginate_list_tickets(include="requester,company"):
        tid = t.get("id")
        try:
            conversations = _paginate_conversations(tid)
            t["conversations"] = conversations
            convo_rows.extend(flatten_conversations(tid, conversations))
        except requests.HTTPError as e:
            t["conversations"] = []
            t["conversations_error"] = f"{getattr(e.response, 'status_code', '?')}: {e.response.text[:200]}" if e.response else "Unknown error"

        ticket_rows.append(flatten_ticket(t))
        count += 1
        if count % 100 == 0:
            print(f"Fetched {count} tickets...", file=sys.stderr)

    # Write tickets CSV
    with open(ticket_csv, "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=ticket_rows[0].keys())
        writer.writeheader()
        writer.writerows(ticket_rows)

    # Write conversations CSV
    if convo_rows:
        with open(convo_csv, "w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=convo_rows[0].keys())
            writer.writeheader()
            writer.writerows(convo_rows)

    return count

# ---- Entry Point ----
if __name__ == "__main__":
    try:
        total = fetch_all_tickets_to_csv("tickets.csv", "conversations.csv")
        print(f"✅ Done. Wrote {total} tickets to tickets.csv and their conversations to conversations.csv")
    except requests.HTTPError as e:
        print("HTTP error:", getattr(e.response, "status_code", "?"), (e.response.text[:500] if e.response else ""))
    except Exception as e:
        print("Error:", e)


✅ Done. Wrote 8 tickets to tickets.csv and their conversations to conversations.csv


In [5]:
#!/usr/bin/env python3
# Fetch ALL Freshdesk tickets + conversations (python-dotenv)
# Outputs:
#   - tickets_full.jsonl         (one ticket per line with conversations)
#   - tickets_references.json    (array of minimal reference objects with links)

import os, sys, time, json
from typing import Dict, Iterable, List, Optional
from urllib.parse import urlparse

import requests
from dotenv import load_dotenv, find_dotenv

# ---- Load .env (falls back to real env if already set) ----
load_dotenv(find_dotenv(usecwd=True), override=False)

# ---- Helpers ----
def require_env(name: str) -> str:
    v = os.getenv(name, "").strip()
    if not v:
        sys.exit(f"Missing env var: {name}")
    return v

def parse_host(domain: str) -> str:
    """
    Accepts 'acme', 'acme.freshdesk.com', or 'https://acme.freshdesk.com'
    -> 'acme.freshdesk.com'
    """
    d = domain.strip()
    if d.lower().startswith("http"):
        parsed = urlparse(d)
        host = (parsed.netloc or parsed.path).strip("/")
    else:
        host = d.strip("/")
    if not host.endswith(".freshdesk.com"):
        host = f"{host}.freshdesk.com"
    return host

def build_bases(domain: str) -> Dict[str, str]:
    host = parse_host(domain)
    return {
        "HOST": host,
        "API_BASE":   f"https://{host}/api/v2",
        "AGENT_BASE": f"https://{host}/a",
    }

DOMAIN   = require_env("FRESHDESK_DOMAIN")
API_KEY  = require_env("FRESHDESK_API_KEY")
TIMEOUT  = int(os.getenv("FRESHDESK_TIMEOUT_SECONDS", "30"))
BASES    = build_bases(DOMAIN)
API_BASE = BASES["API_BASE"]
AGENT_BASE = BASES["AGENT_BASE"]

session = requests.Session()
session.auth = (API_KEY, "X")  # Freshdesk: API key as username, 'X' as password
session.headers.update({"Content-Type": "application/json"})

def _request(method: str, url: str, params: Optional[Dict]=None) -> requests.Response:
    """Basic 429/5xx retry with exponential backoff."""
    backoff = 1
    while True:
        resp = session.request(method, url, params=params, timeout=TIMEOUT)
        if resp.status_code == 429:
            retry_after = int(resp.headers.get("Retry-After", backoff))
            time.sleep(max(1, retry_after))
            backoff = min(backoff * 2, 30)
            continue
        if 500 <= resp.status_code < 600:
            if backoff > 30:
                resp.raise_for_status()
            time.sleep(backoff)
            backoff = min(backoff * 2, 30)
            continue
        resp.raise_for_status()
        return resp

def _paginate_list_tickets(include: str = "requester,company", per_page: int = 100) -> Iterable[Dict]:
    """
    Iterate over ALL tickets using page/per_page.
    include=requester,company embeds requester/company details for each ticket.
    """
    page = 1
    while True:
        params = {"page": page, "per_page": per_page}
        if include:
            params["include"] = include
        resp = _request("GET", f"{API_BASE}/tickets", params=params)
        items = resp.json() if resp.text else []
        if not items:
            break
        for t in items:
            yield t
        page += 1

def _paginate_conversations(ticket_id: int, per_page: int = 100) -> List[Dict]:
    """
    Fetch all conversations (agent replies, requester replies, and notes).
    """
    out: List[Dict] = []
    page = 1
    while True:
        params = {"page": page, "per_page": per_page}
        resp = _request("GET", f"{API_BASE}/tickets/{ticket_id}/conversations", params=params)
        items = resp.json() if resp.text else []
        if not items:
            break
        out.extend(items)
        page += 1
    return out

# ---- Reference link builders ----
def ticket_links(ticket_id: int) -> Dict[str, str]:
    return {
        "reference_link": f"{AGENT_BASE}/tickets/{ticket_id}",
        "api_link": f"{API_BASE}/tickets/{ticket_id}",
        "conversations_api_link": f"{API_BASE}/tickets/{ticket_id}/conversations",
    }

def contact_links(contact_id: int) -> Dict[str, str]:
    return {
        "requester_reference_link": f"{AGENT_BASE}/contacts/{contact_id}",
        "requester_api_link": f"{API_BASE}/contacts/{contact_id}",
    }

def company_links(company_id: int) -> Dict[str, str]:
    return {
        "company_reference_link": f"{AGENT_BASE}/companies/{company_id}",
        "company_api_link": f"{API_BASE}/companies/{company_id}",
    }

def extract_attachment_refs(conversations: List[Dict]) -> List[Dict]:
    """
    Collect minimal attachment references from conversations.
    Each item: {name, size, content_type, url}
    """
    refs: List[Dict] = []
    for conv in conversations or []:
        atts = conv.get("attachments") or []
        if isinstance(atts, list):
            for a in atts:
                # Freshdesk typically provides 'attachment_url' (or sometimes 'url')
                url = a.get("attachment_url") or a.get("url")
                refs.append({
                    "name": a.get("name"),
                    "size": a.get("size"),
                    "content_type": a.get("content_type"),
                    "url": url,
                })
    return refs

def build_ticket_reference(ticket: Dict, conversations: Optional[List[Dict]]) -> Dict:
    tid = int(ticket["id"])
    ref = {"id": tid, **ticket_links(tid)}
    # optional requester / company links
    rid = ticket.get("requester_id")
    if rid:
        ref.update(contact_links(int(rid)))
    cid = ticket.get("company_id")
    if cid:
        ref.update(company_links(int(cid)))
    # attachments discovered from conversations
    if conversations is not None:
        ref["attachments"] = extract_attachment_refs(conversations)
    return ref

# ---- Fetch & write ----
def fetch_all_tickets_with_details(
    outfile: str = "tickets_full.jsonl",
    refs_outfile: str = "tickets_references.json"
) -> int:
    """
    Streams tickets + conversations to a JSONL file,
    and writes a reference JSON array with links for each ticket.
    Returns count of tickets written.
    """
    count = 0
    with open(outfile, "w", encoding="utf-8") as f, open(refs_outfile, "w", encoding="utf-8") as rf:
        # Start streaming array for references to avoid high memory
        rf.write("[")
        first_ref = True

        for t in _paginate_list_tickets(include="requester,company"):
            tid = t.get("id")
            conversations: Optional[List[Dict]] = None
            try:
                conversations = _paginate_conversations(tid)
                t["conversations"] = conversations
            except requests.HTTPError as e:
                # Keep going even if conversations fail on a ticket
                t["conversations_error"] = {
                    "status": getattr(e.response, "status_code", None),
                    "body": (e.response.text[:300] if getattr(e, "response", None) else "")
                }

            # Write full ticket line
            f.write(json.dumps(t, ensure_ascii=False) + "\n")

            # Build and stream reference object
            ref_obj = build_ticket_reference(t, conversations)
            chunk = json.dumps(ref_obj, ensure_ascii=False)
            if first_ref:
                rf.write(chunk)
                first_ref = False
            else:
                rf.write("," + chunk)

            count += 1
            if count % 100 == 0:
                print(f"Wrote {count} tickets...", file=sys.stderr)

        rf.write("]")  # close JSON array

    return count

# ---- Main ----
if __name__ == "__main__":
    try:
        total = fetch_all_tickets_with_details(
            outfile="tickets_full.jsonl",
            refs_outfile="tickets_references.json"
        )
        print(f"✅ Done.")
        print(f"   • Tickets: {total} -> tickets_full.jsonl")
        print(f"   • References JSON: tickets_references.json")
    except requests.HTTPError as e:
        print("HTTP error:", getattr(e.response, "status_code", "?"), (e.response.text[:500] if e.response else ""))
    except Exception as e:
        print("Error:", e)


✅ Done.
   • Tickets: 8 -> tickets_full.jsonl
   • References JSON: tickets_references.json
